In [ ]:
from huggingface_hub import login
token = ""
print("Hugging Face logging")
login(token)

Hugging Face logging


In [ ]:
import torch
# No se usa mps porque da problemas en el notebook
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# Corpus

In [ ]:

def get_boe_dispositions(query):
    # URL base para realizar la búsqueda en el BOE
    #url = f"https://www.boe.es/buscar/legislacion.php?campo%5B2%5D=TITULOS&dato%5B2%5D={query}&accion=Buscar"
    url = f"https://www.boe.es/buscar/boe.php?campo%5B0%5D=ORIS&dato%5B0%5D%5B1%5D=1&dato%5B0%5D%5B2%5D=2&dato%5B0%5D%5B3%5D=3&dato%5B0%5D%5B4%5D=4&dato%5B0%5D%5B5%5D=5&dato%5B0%5D%5BT%5D=T&operador%5B0%5D=and&campo%5B1%5D=TITULOS&dato%5B1%5D={query}&operador%5B1%5D=and&campo%5B2%5D=DEM&dato%5B2%5D=&operador%5B2%5D=and&campo%5B3%5D=DOC&dato%5B3%5D=&operador%5B3%5D=and&campo%5B4%5D=NBOS&dato%5B4%5D=&operador%5B4%5D=and&campo%5B5%5D=NOF&dato%5B5%5D=&operador%5B5%5D=and&operador%5B6%5D=and&campo%5B6%5D=FPU&dato%5B6%5D%5B0%5D=&dato%5B6%5D%5B1%5D=&page_hits=50&sort_field%5B0%5D=FPU&sort_order%5B0%5D=desc&sort_field%5B1%5D=ORI&sort_order%5B1%5D=asc&sort_field%5B2%5D=REF&sort_order%5B2%5D=asc&accion=Buscar"

    # Realizar la solicitud a la página
    response = requests.get(url)
    if response.status_code == 200:
        # Parsear el HTML de la página
        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar las disposiciones dentro del HTML
        results = soup.find_all("li", class_="resultado-busqueda")

        dispositions = []
        for result in results[:10]:
            # Extraer el título
            title_tag = result.find("h3")
            title = title_tag.get_text(strip=True) if title_tag else "Título no disponible"

            # Extraer el resumen
            summary_tag = result.find("p")
            summary = summary_tag.get_text(strip=True) if summary_tag else "Resumen no disponible"

            # Extraer el enlace y completar la URL
            link_tag = result.find("a", class_="resultado-busqueda-link-defecto")
            link = "https://www.boe.es" + link_tag["href"].replace('..', '') if link_tag else "Enlace no disponible"

            # Extraer el texto y los metadatos
            [resultTXT, result_publicado_en, result_seccion, result_departamento, result_referencia] = get_boe_txt_metadata(link)

            # Guardar la información
            dispositions.append({"title": title, "summary": summary, "url": link, "texto": resultTXT, "publicado": result_publicado_en, "seccion": result_seccion, "departamento": result_departamento, "referencia": result_referencia})

        return dispositions
    else:
        print(f"Error al acceder al BOE: {response.status_code}")
        return []

def get_boe_info_metadata(result, typeInfo):
    info_tag = result.find("dt", string = typeInfo)
    if info_tag:
      dd_tag = info_tag.find_next_sibling("dd")
      if dd_tag:
          info = dd_tag.get_text(strip=True)
      else:
          info = "Etiqueta <dd> no encontrada"
    else:
        info = f"Etiqueta <dt> '{typeInfo}' no encontrada"

    return info

def get_boe_metadata(soup):
  resultsMetadata = soup.find_all("div", class_="metadatosDoc")
  for result in resultsMetadata:
    publicado_en = get_boe_info_metadata(result, "Publicado en:")
    seccion = get_boe_info_metadata(result, "Sección:")
    departamento = get_boe_info_metadata(result, "Departamento:")
    referencia = get_boe_info_metadata(result, "Referencia:")

  return [publicado_en, seccion, departamento, referencia]

def get_boe_txt_metadata(url):
    metadatas = []

    response = requests.get(url)
    if response.status_code == 200:
      # Parsear el HTML de la página
      soup = BeautifulSoup(response.content, "html.parser")
      resultTXT = soup.find_all("div", id="DOdocText")
      if (len(resultTXT) > 0):
        resultTXT = resultTXT[0]
      else:
        resultTXT = ""

      [result_publicado_en, result_seccion, result_departamento, result_referencia] = get_boe_metadata(soup)
      return [resultTXT, result_publicado_en, result_seccion, result_departamento, result_referencia]
    else:
      print(f"Error al acceder al BOE: {response.status_code}")
      return []

def remove_html_tags(text):
    """Removes HTML tags from a string."""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
# Probar con una consulta específica
query = "discapacidad"
boe_dispositions = get_boe_dispositions(query)

# Mostrar resultados
for i, disposition in enumerate(boe_dispositions, 1):
    texto = disposition['texto'].get_text()
    print(f"{i}. {disposition['title']}")
    print(f"   Resumen: {disposition['summary']}")
    print(f"   URL: {disposition['url']}")
    print(f"   Publicado en: {disposition['publicado']}")
    print(f"   Sección: {disposition['seccion']}")
    print(f"   Departamento: {disposition['departamento']}")
    print(f"   Referencia: {disposition['referencia']}")
    print(f"   Texto: {texto[:50]}\n")

1. Ministerio de Derechos Sociales, Consumo y Agenda 2030
   Resumen: Anuncio de formalización de contratos de: Subdirección General de Análisis Presupuestario y Gestión Financiera del Instituto de Mayores y Servicios Sociales . Objeto: Suministro de víveres con destino al Centro de Recuperación de Personas con Discapacidad Física sito en calle Donantes de Sangre, 1 de Lardero (La Rioja). Expediente: 651/2024.
   URL: https://www.boe.es/buscar/doc.php?id=BOE-B-2025-3065
   Publicado en: «BOE»núm.26, de 30 de enero de 2025, páginas 4750 a 4753 (4págs.)
   Sección: V. Anuncios- A. Contratación del Sector Público
   Departamento: Ministerio de Derechos Sociales, Consumo y Agenda 2030
   Referencia: BOE-B-2025-3065
   Texto: 
TEXTO


1. Poder adjudicador: 


1.1) Nombre: 
Su

2. Ministerio de Derechos Sociales, Consumo y Agenda 2030
   Resumen: Anuncio de formalización de contratos de: Subdirección General de Análisis Presupuestario y Gestión Financiera del Instituto de Mayores y Servicios

In [ ]:
cadena_busqueda = "busca donde aparece regulado el porcentaje de discapacitados que deben contratar las empresas"